In [1]:
import boto3

from urllib.request import urlopen
from PIL import Image 

import pandas as pd
import json

In [2]:
df = pd.read_csv('../data/data_including_subscriber_and_thumbnail.csv')
df = df[['video_id', 'views', 'subscriber_count', 'thumbnail']]
df

,video_id,views,subscriber_count,thumbnail
0,8e7GcsDEKEE,30038.0,98400,https://i.ytimg.com/vi/8e7GcsDEKEE/default.jpg
1,FYoDak3VfF4,29102.0,98400,https://i.ytimg.com/vi/FYoDak3VfF4/default.jpg
2,FuTTOzwWo58,25022.0,98400,https://i.ytimg.com/vi/FuTTOzwWo58/default.jpg
3,-48XAC_GDjk,49551.0,98400,https://i.ytimg.com/vi/-48XAC_GDjk/default.jpg
4,mhehORE9WNA,74348.0,98400,https://i.ytimg.com/vi/mhehORE9WNA/default.jpg
...,...,...,...,...
101159,K-sH6E9YeTw,137922.0,112000,https://i.ytimg.com/vi/K-sH6E9YeTw/default.jpg
101160,ucEs84amVm8,153005.0,112000,https://i.ytimg.com/vi/ucEs84amVm8/default.jpg
101161,jSK3H-dFfE0,151071.0,112000,https://i.ytimg.com/vi/jSK3H-dFfE0/default.jpg
101162,OEWk-sZG5j0,177515.0,112000,https://i.ytimg.com/vi/OEWk-sZG5j0/default.jpg


In [3]:
def detect_faces(url):

    client=boto3.client('rekognition')

    response = client.detect_faces(
        Image={'Bytes': urlopen(url).read()},
        Attributes=['ALL']
    )

    for faceDetail in response['FaceDetails']:
        # age는 예상 나이 범위의 (low+high)/2
        age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) / 2
        
        # size
        width = faceDetail['BoundingBox']['Width']
        height = faceDetail['BoundingBox']['Height']
        
        # emotion
        emotion_confidence = 0
        emotion = ''
        
        for item in faceDetail['Emotions']:
            if item['Confidence'] > emotion_confidence:
                emotion_confidence = item['Confidence']
                emotion = item['Type']
        
        # gender
        gender = faceDetail['Gender']['Value']
        
        data = [age, width, height, emotion, gender]
    return data

In [4]:
def get_data(start_index, end_index):
    if end_index > len(df):
        end_index = len(df)
    for index in range(start_index, end_index):
        url = df['thumbnail'][index]
        data = detect_faces(url)
        df.loc[index, ['age', 'width', 'height', 'emotion', 'gender']] = data

In [5]:
get_data(0,10)

df.head(15)

,video_id,views,subscriber_count,thumbnail,age,width,height,emotion,gender
0,8e7GcsDEKEE,30038.0,98400,https://i.ytimg.com/vi/8e7GcsDEKEE/default.jpg,39.0,0.163759,0.265995,SAD,Male
1,FYoDak3VfF4,29102.0,98400,https://i.ytimg.com/vi/FYoDak3VfF4/default.jpg,9.0,0.209445,0.350828,SURPRISED,Male
2,FuTTOzwWo58,25022.0,98400,https://i.ytimg.com/vi/FuTTOzwWo58/default.jpg,18.0,0.210325,0.342480,CALM,Male
3,-48XAC_GDjk,49551.0,98400,https://i.ytimg.com/vi/-48XAC_GDjk/default.jpg,22.0,0.143989,0.230920,CALM,Male
4,mhehORE9WNA,74348.0,98400,https://i.ytimg.com/vi/mhehORE9WNA/default.jpg,28.0,0.152964,0.214099,CALM,Male
5,hBceqD0_fdI,28711.0,98400,https://i.ytimg.com/vi/hBceqD0_fdI/default.jpg,20.0,0.149496,0.251399,CALM,Female
6,lkyC67fwjVw,29543.0,98400,https://i.ytimg.com/vi/lkyC67fwjVw/default.jpg,18.0,0.261811,0.410678,SAD,Male
7,1Sgj_6U4esI,53219.0,98400,https://i.ytimg.com/vi/1Sgj_6U4esI/default.jpg,25.0,0.170436,0.269196,HAPPY,Male
8,BALd4uDNKGk,64239.0,98400,https://i.ytimg.com/vi/BALd4uDNKGk/default.jpg,28.0,0.267067,0.418738,SAD,Female
9,te3Fw28PnG4,70823.0,98400,https://i.ytimg.com/vi/te3Fw28PnG4/default.jpg,17.0,0.205526,0.358913,SAD,Male


In [6]:
df.to_csv('../data/face_dectection_data.csv', index=False)